In [1]:
import torch
print(torch.cuda.is_available())

True


In [2]:
import huggingface_hub
huggingface_hub.login()

In [1]:
from unsloth import FastModel
import torch 

model, tokenizer = FastModel.from_pretrained(
    model_name = "MailMindActionItems4",
    max_seq_length = 2046,
    load_in_4bit= True,
    load_in_8bit= False,
    full_finetuning= False,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


c:\repos\MailMind\.venv\Lib\site-packages\unsloth_zoo\gradient_checkpointing.py:330: UserWarning: expandable_segments not supported on this platform (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\c10/cuda/CUDAAllocatorConfig.h:28.)
  GPU_BUFFERS = tuple([torch.empty(2*256*2048, dtype = dtype, device = f"cuda:{i}") for i in range(n_gpus)])


==((====))==  Unsloth 2025.3.19: Fast Gemma3 patching. Transformers: 4.50.0.dev0.
   \\   /|    NVIDIA GeForce RTX 2060. Num GPUs = 1. Max memory: 6.0 GB. Platform: Windows.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Using float16 precision for gemma3 won't work! Using float32.


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.48, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [ ]:
model = FastModel.get_peft_model(
    model,
    finetune_vision_layers     = False, # Turn off for just text!
    finetune_language_layers   = True,  # Should leave on!
    finetune_attention_modules = True,  # Attention good for GRPO
    finetune_mlp_modules       = True,  # SHould leave on always!

    r = 64,           # Larger = higher accuracy, but might overfit
    lora_alpha = 64,  # Recommended alpha == r at least
    lora_dropout = 0,
    bias = "none",
    random_state = 3407,
)

In [2]:
from unsloth.chat_templates import get_chat_template
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "gemma-3",
)

In [3]:
from datasets import load_dataset
dataset = load_dataset("DenverDawgs18/ActionItems", split="train")

In [4]:
from unsloth.chat_templates import standardize_data_formats
dataset = standardize_data_formats(dataset)

In [9]:
print(dataset[100])

{'Unnamed: 0': 100, 'Body': 'Looking forward to our discussion.  ', 'Action_Item': 'No action'}


In [5]:
def apply_chat_template(examples):
    formatted_conversations = []
    
    for idx in range(len(examples["Body"])):
        try:
            # Create a proper instruction from the email
            instruction = f"Provide the action item(s) for this email. Action items are defined as things the sender of the email expects the reciever to complete, such as questions and requests. You should try to keep the language the same (for example if it says Tuesday don't return Tues.). Many emails do not have action items, in this case, return No action. Huge mailing lists should never have action items. You need to keep in mind that emails being forwarded or replies to emails that had action items doesn't necessarily mean that the reciever needs to do what the original email says. You should use context to figure that out. \n\n{examples['Body'][idx]}"
            
            # Create a conversation structure for Gemma's chat template
            conversation = [
                {"role": "user", "content": instruction},
                {"role": "assistant", "content": examples["Action_Item"][idx]}
            ]
            
            # Apply the chat template to the conversation
            formatted_text = tokenizer.apply_chat_template(conversation)
            formatted_conversations.append(formatted_text)
        except Exception as e:
            print(f"Error processing example {idx}: {e}")
            # Add a default value for failed processing
            formatted_conversations.append("")
    
    return {"text": formatted_conversations}
dataset = dataset.map(apply_chat_template, batched = True, num_proc=1)
print(dataset[100]["text"])

<bos><start_of_turn>user
Provide the action item(s) for this email. Action items are defined as things the sender of the email expects the reciever to complete, such as questions and requests. You should try to keep the language the same (for example if it says Tuesday don't return Tues.). Many emails do not have action items, in this case, return No action. Huge mailing lists should never have action items. You need to keep in mind that emails being forwarded or replies to emails that had action items doesn't necessarily mean that the reciever needs to do what the original email says. You should use context to figure that out. 

Looking forward to our discussion.<end_of_turn>
<start_of_turn>model
No action<end_of_turn>



In [6]:
from trl import SFTTrainer, SFTConfig
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    eval_dataset = None, # Can set up evaluation!
    args = SFTConfig(
        dataset_text_field = "text",
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4, # Use GA to mimic batch size!
        warmup_steps = 5,
        num_train_epochs = 10,
        max_steps = 6,
        learning_rate = 2e-4, # Reduce to 2e-5 for long training runs
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        report_to = "none", # Use this for WandB etc
        dataset_num_proc= 1,
    ),
)

Unsloth: Switching to float32 training since model cannot work with float16


In [7]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<start_of_turn>user\n",
    response_part = "<start_of_turn>model\n",
)

In [7]:
print(tokenizer.decode(trainer.train_dataset[100]["input_ids"]))

<bos><bos><start_of_turn>user
Provide the action item(s) for this email. Action items are defined as things the sender of the email expects the reciever to complete, such as questions and requests. You should try to keep the language the same (for example if it says Tuesday don't return Tues.). Many emails do not have action items, in this case, return No action. Huge mailing lists should never have action items. You need to keep in mind that emails being forwarded or replies to emails that had action items doesn't necessarily mean that the reciever needs to do what the original email says. You should use context to figure that out. 

Looking forward to our discussion.<end_of_turn>
<start_of_turn>model
No action<end_of_turn>



In [9]:
print(tokenizer.decode([tokenizer.pad_token_id if x == -100 else x for x in trainer.train_dataset[100]["labels"]]).replace(tokenizer.pad_token, " "))

                                                                                                                                                 No action<end_of_turn>



In [8]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 1,250 | Num Epochs = 1 | Total steps = 6
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 119,209,984/4,000,000,000 (2.98% trained)
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,0.469300
2,0.922800
3,0.277900
4,0.135400
5,0.271000
6,0.065600


In [15]:
from unsloth.chat_templates import get_chat_template
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "gemma-3",
)
messages = [{
    "role": "user",
    "content": [{
        "type" : "text",
        "text" : "Continue the sequence: 1, 1, 2, 3, 5, 8,",
    }]
}]
text = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True, # Must add for generation
)
outputs = model.generate(
    **tokenizer([text], return_tensors = "pt").to("cuda"),
    max_new_tokens = 64,  # Increase for longer outputs!
    # Recommended Gemma-3 settings!
    temperature = 1.0, top_p = 0.95, top_k = 64,
)
tokenizer.batch_decode(outputs)

['<bos><start_of_turn>user\nContinue the sequence: 1, 1, 2, 3, 5, 8,<end_of_turn>\n<start_of_turn>model\n13<end_of_turn>']

In [9]:
model.save_pretrained("MailMindActionItems5")
tokenizer.save_pretrained("MailMindActionItems5")

['MailMindActionItems5\\processor_config.json']

In [3]:

from unsloth import FastModel
model, tokenizer = FastModel.from_pretrained(
        model_name = "MailMindActionItems5", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = 2048,
        load_in_4bit = True,
        device_map="cuda:0"
        
    )

messages = [{
    "role": "user",
    "content": [{"type" : "text", "text" : f"Provide the action item(s) for this email using context if needed. If not, return No action. Actionable items include questions, requests for comments, and requests for files. {dataset[100]['email']}",}]
}]
text = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True, # Must add for generation
)

from transformers import TextStreamer
_ = model.generate(
    **tokenizer([text], return_tensors = "pt").to("cuda"),
    max_new_tokens = 256, # Increase for longer outputs!
    # Recommended Gemma-3 settings!
    temperature = 1.0, top_p = 0.95, top_k = 64,
    streamer = TextStreamer(tokenizer, skip_prompt = True),
)

==((====))==  Unsloth 2025.3.19: Fast Gemma3 patching. Transformers: 4.50.0.dev0.
   \\   /|    NVIDIA GeForce RTX 2060. Num GPUs = 1. Max memory: 6.0 GB. Platform: Windows.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Using float16 precision for gemma3 won't work! Using float32.


KeyError: 'email'

In [3]:
from unsloth import FastLanguageModel
from datasets import load_dataset
import evaluate
import torch
from tqdm import tqdm
from collections import defaultdict
import os 
import tempfile


# === Load your fine-tuned Unsloth model ===
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="MailMindActionItems5",
    max_seq_length=2048,
    load_in_4bit=True,
    device_map="cuda:0"
)

# === Load the dataset ===
dataset = load_dataset("DenverDawgs18/ActionItems", split="test")

# === Prompt template ===
def format_prompt(body):
    return (
        "<|system|>Provide the action item(s) for this email. Action items are defined as things the sender of the email expects the receiver to complete, such as questions and requests. You should try to keep the language the same (for example if it says Tuesday don't return Tues.). Many emails do not have action items, in this case, return No action. Huge mailing lists should never have action items. You need to keep in mind that emails being forwarded or replies to emails that had action items doesn't necessarily mean that the receiver needs to do what the original email says. You should use context to figure that out.<|user|>\n" + body + "\n<|assistant|>"
    )

# === Generation config ===
gen_kwargs = {
    "max_new_tokens": 140,
    "do_sample": False,
    "temperature": 1,
    "top_p": 0.9,
}

# === Initialize tracking ===
all_results = []
tp_results = []  # store results for TPs only

rouge = evaluate.load("rouge")
dataset = list(dataset)

TP = FP = FN = TN = 0

for i, item in enumerate(tqdm(dataset)):
    prompt = format_prompt(item["Body"])
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

    with torch.no_grad():
        outputs = model.generate(**inputs, **gen_kwargs)

    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
    prediction = decoded.split("<|assistant|>")[-1].strip()
    reference = item["Action_Item"]
    result = rouge.compute(predictions=[prediction], references=[reference])
    # Determine if action items exist
    is_ref_action = reference.strip().lower() != "no action" and reference.strip().lower() != "no action."
    is_pred_action = prediction.strip().lower() != "no action" and prediction.strip().lower() != "no action."

    if is_ref_action and is_pred_action:
        TP += 1
        tp_results.append(result)
    elif not is_ref_action and is_pred_action:
        FP += 1
    elif is_ref_action and not is_pred_action:
        FN += 1
    else:
        TN += 1

    # Always compute ROUGE for overall stats
    if result["rouge1"] <= 0.61:
        print("Index: ", i)
        print('Body: ', item["Body"])
        print("Reference: ", item["Action_Item"])
        print("Predicted: ", prediction)
        for key, value in result.items():
            print(f"{key}: {value:.4f}")

    all_results.append(result)

    # === Intermediate reporting every 10 ===
    if i % 10 == 0:
        avg_scores = defaultdict(float)
        for result in all_results:
            for key, value in result.items():
                avg_scores[key] += value
        for key in avg_scores:
            avg_scores[key] /= len(all_results)

        tp_avg_scores = defaultdict(float)
        if tp_results:
            for result in tp_results:
                for key, value in result.items():
                    tp_avg_scores[key] += value
            for key in tp_avg_scores:
                tp_avg_scores[key] /= len(tp_results)

        # Detection performance
        precision = TP / (TP + FP) if (TP + FP) > 0 else 0
        recall = TP / (TP + FN) if (TP + FN) > 0 else 0
        f1 = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

        # === Report ===
        print("\n===== METRICS =====")
        print("Overall Average ROUGE:")
        for key, value in avg_scores.items():
            print(f"{key}: {value:.4f}")

        if tp_results:
            print("\nTrue Positive ROUGE:")
            for key, value in tp_avg_scores.items():
                print(f"{key}: {value:.4f}")

        print("\nDetection performance:")
        print(f"TP: {TP}, FP: {FP}, FN: {FN}, TN: {TN}")
        print(f"Precision: {precision:.4f}")
        print(f"Recall:    {recall:.4f}")
        print(f"F1 Score:  {f1:.4f}")


==((====))==  Unsloth 2025.3.19: Fast Gemma3 patching. Transformers: 4.50.0.dev0.
   \\   /|    NVIDIA GeForce RTX 2060. Num GPUs = 1. Max memory: 6.0 GB. Platform: Windows.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Using float16 precision for gemma3 won't work! Using float32.


OutOfMemoryError: CUDA out of memory. Tried to allocate 1.28 GiB. GPU 0 has a total capacity of 6.00 GiB of which 0 bytes is free. Of the allocated memory 20.26 GiB is allocated by PyTorch, and 68.04 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)